In [43]:
import pandas as pd

# Load Enhance and Live Price Data
enhance_df = pd.read_csv("enhanced_stock_snapshot.csv")
live_price_df = pd.read_csv("live_prices/live_prices_2025-07-18.csv")

# Standardize column names
enhance_df.columns = enhance_df.columns.str.strip()
live_price_df.columns = live_price_df.columns.str.strip()

# Preview column names to identify matching keys
print("Enhance Columns:", enhance_df.columns.tolist())
print("Live Price Columns:", live_price_df.columns.tolist())

# Merge the data on STOCK (enhance) == SYMBOL (live)
merged_df = pd.merge(
    enhance_df,
    live_price_df[['SYMBOL', 'PRICE']],  # Only need SYMBOL and PRICE
    left_on='STOCK',
    right_on='SYMBOL',
    how='left'  # Use 'left' to keep all Enhance rows
)

# Drop duplicate SYMBOL column
merged_df.drop(columns=['SYMBOL'], inplace=True)

# Save or display merged data
print("✅ Merged data sample:")
print(merged_df.head())

# Optionally save
merged_df.to_csv("Merged_Enhance_with_LivePrice.csv", index=False)


Enhance Columns: ['DATE', 'CLOSE', 'HIGH', 'LOW', 'OPEN', 'VOLUME', 'STOCK', 'daily_return', 'volatility_30d', 'avg_return_30d', 'risk_level']
Live Price Columns: ['SYMBOL', 'PRICE', 'FETCHED_AT']
✅ Merged data sample:
         DATE         CLOSE          HIGH           LOW          OPEN  \
0  2025-07-17    255.009995    259.899994    251.419998    259.899994   
1  2025-07-17     59.169998     59.180000     58.000000     59.000000   
2  2025-07-17   1205.599976   1209.699951   1189.300049   1205.000000   
3  2025-07-17     25.629999     26.450001     25.440001     25.740000   
4  2025-07-17  30000.000000  30260.000000  29870.000000  30180.000000   

    VOLUME       STOCK  daily_return  volatility_30d  avg_return_30d  \
0   517794   20MICRONS     -0.018248        0.035302        0.004112   
1    12885  21STCENMGM      0.019821        0.015860       -0.003543   
2   484846      360ONE      0.006680        0.020044        0.004052   
3  1061963   3IINFOLTD     -0.001558        0.022311  

In [31]:
import pandas as pd

df = pd.read_csv("mutual_fund_volatility_report.csv")
df = df.rename(columns={"Risk Category": "Risk_Category"})
df.to_csv("mutual_fund_volatility_report1.csv", index=False)


In [3]:
import pandas as pd

# Step 1: Load the files
merged_df = pd.read_csv("Merged_Enhance_with_LivePrice.csv")  # stock data
mf_df = pd.read_csv("mutual_fund_volatility_report1.csv")      # mutual fund data
customer_df = pd.read_csv("data2/customer_information.csv")    # customer data

# Step 2: Clean and fill nulls
merged_df.fillna({
    "volatility_30d": merged_df["volatility_30d"].mean(),
    "avg_return_30d": merged_df["avg_return_30d"].mean(),
    "risk_level": "Medium"
}, inplace=True)

mf_df.rename(columns={
    "Volatility (90d)": "volatility_30d",
    "Return (90d%)": "avg_return_30d"
}, inplace=True)

mf_df.fillna({
    "volatility_30d": mf_df["volatility_30d"].mean(),
    "avg_return_30d": mf_df["avg_return_30d"].mean(),
    "Risk_Category": "Medium Risk"
}, inplace=True)

# Step 3: Add common columns
merged_df["instrument_name"] = merged_df["STOCK"]
merged_df["instrument_type"] = "Stock"
merged_df["Risk_Category"] = merged_df["risk_level"]

mf_df["instrument_name"] = mf_df["Scheme Name"]
mf_df["instrument_type"] = "Mutual Fund"

# Step 4: Select required columns
stock_reco = merged_df[["instrument_name", "volatility_30d", "avg_return_30d", "instrument_type", "Risk_Category"]]
mf_reco = mf_df[["instrument_name", "volatility_30d", "avg_return_30d", "instrument_type", "Risk_Category"]]

# Step 5: Combine both
recommendations = pd.concat([stock_reco, mf_reco], ignore_index=True)

# Step 6: Save or print
recommendations.to_csv("final_recommendations.csv", index=False)
print(f"✅ Final recommendation file generated. Rows: {len(recommendations)}")


✅ Final recommendation file generated. Rows: 9975


In [ ]:
import pandas as pd

# Step 1: Load your data
customer_df = pd.read_csv("data2/customer_information.csv")
stock_df = pd.read_csv("Merged_Enhance_with_LivePrice.csv")
mf_df = pd.read_csv("mutual_fund_volatility_report1.csv")

# Step 2: Merge stock and mutual fund recommendations
base_recommendations = pd.concat([stock_df, mf_df], ignore_index=True)

# Step 3: Define output CSV file
output_file = "final_recommendations.csv"

# Step 4: Initialize the CSV with headers (only once)
base_recommendations_copy = base_recommendations.copy()
base_recommendations_copy["customerID"] = ""  # Add empty column to define structure
base_recommendations_copy.head(0).to_csv(output_file, index=False)

# Step 5: Write customer-specific rows one-by-one (streaming)
for idx, customer in customer_df.iterrows():
    customer_id = customer["customerID"]
    
    # Copy and assign customerID
    recs = base_recommendations.copy()
    recs["customerID"] = customer_id
    
    # Append to CSV (no header after first write)
    recs.to_csv(output_file, mode='a', header=False, index=False)

    if idx % 50 == 0:
        print(f"✅ Processed {idx+1}/{len(customer_df)} customers")

print("🎉 Final recommendations saved to:", output_file)


✅ Processed 1/32468 customers
✅ Processed 51/32468 customers
✅ Processed 101/32468 customers
✅ Processed 151/32468 customers
✅ Processed 201/32468 customers
✅ Processed 251/32468 customers
✅ Processed 301/32468 customers
✅ Processed 351/32468 customers
✅ Processed 401/32468 customers
✅ Processed 451/32468 customers
✅ Processed 501/32468 customers
✅ Processed 551/32468 customers
✅ Processed 601/32468 customers
✅ Processed 651/32468 customers
✅ Processed 701/32468 customers
✅ Processed 751/32468 customers
✅ Processed 801/32468 customers
✅ Processed 851/32468 customers
✅ Processed 901/32468 customers
✅ Processed 951/32468 customers
✅ Processed 1001/32468 customers
✅ Processed 1051/32468 customers
✅ Processed 1101/32468 customers
✅ Processed 1151/32468 customers
✅ Processed 1201/32468 customers
✅ Processed 1251/32468 customers
✅ Processed 1301/32468 customers
✅ Processed 1351/32468 customers
✅ Processed 1401/32468 customers
✅ Processed 1451/32468 customers
✅ Processed 1501/32468 customers


In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Step 1: Load the CSV (adjust the file path as needed)
csv_path = "C:/Users/mruna/Downloads/final_recommendations_with_customerID.csv"  # Full path if required
df = pd.read_csv(csv_path)



In [33]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Your variables
database = "finance_data"     #  Replace with the name you want to create/use
user = "postgres"
password = "root"                   #  Replace with your actual PostgreSQL password
host = "localhost"

# Step 1: Connect to default DB and create your database if it doesn't exist
try:
    conn = psycopg2.connect(
        dbname="postgres",  # connect to default postgres DB first
        user=user,
        password=password,
        host=host
    )
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = %s", (database,))
    exists = cursor.fetchone()
    if not exists:
        cursor.execute(f"CREATE DATABASE {database}")
        print(f"Database '{database}' created.")
    else:
        print(f"ℹ️ Database '{database}' already exists.")
    cursor.close()
    conn.close()
except Exception as e:
    print(f" Error while creating database: {e}")

# Step 2: Connect using SQLAlchemy to upload DataFrame
try:
    # SQLAlchemy engine (use 'postgresql+psycopg2' dialect)
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")

    # Sample DataFrame (replace this with your actual df)
    df = pd.DataFrame({
        'customer_id': [101, 102, 103],
        'recommendation': ['Stock A', 'Mutual Fund B', 'Stock C']
    })

    # Upload the DataFrame to PostgreSQL
    df.to_sql(name='recommendations', con=engine, if_exists='replace', index=False)

    print(" Data uploaded to PostgreSQL successfully.")

except Exception as e:
    print(f"Error while uploading DataFrame: {e}")


✅ Database 'finance_data' created.
✅ Data uploaded to PostgreSQL successfully.
